# Document Classification with Naive Bayes - Lab

## Introduction

In this lecture, you'll practice implementing the Naive Bayes algorithm on your own.

## Objectives

You will be able to:  

* Implement document classification using naive Bayes
* Understand the need for the Laplacian smoothing correction
* Explain how to code a bag of words representation

## Import the Dataset

To start, import the dataset stored in the text file `SMSSpamCollection`.

In [32]:
#Your code here
import pandas as pd

df = pd.read_csv('SMSSpamCollection', delimiter="\t", header=None)
df.columns = ["label", "text"]
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Account for Class Imbalance

To help your algorithm perform more accurately, subset the dataset so that the two classes are of equal size. To do this, keep all of the instances of the minority class (spam) and subset examples of the majority class (ham) to an equal number of examples.

In [33]:
#Your code here
spam_df = df[df['label'] == 'spam']
ham_df = df[df['label'] == 'ham']

len(spam_df), len(ham_df)

(747, 4825)

In [34]:
ham_df = ham_df[:len(spam_df)]
len(ham_df)

747

In [35]:
data = pd.concat([spam_df, ham_df], axis=0, ignore_index=True)
data.head()

,label,text
0,spam,Free entry in 2 a wkly comp to win FA Cup fina...
1,spam,FreeMsg Hey there darling it's been 3 week's n...
2,spam,WINNER!! As a valued network customer you have...
3,spam,Had your mobile 11 months or more? U R entitle...
4,spam,"SIX chances to win CASH! From 100 to 20,000 po..."


## Train - Test Split

Now implement a train test split on your dataset.

In [36]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(data, train_size=.33, random_state=12)
train_df.head()

/opt/conda/envs/learn-env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


,label,text
797,ham,Its a part of checking IQ
1296,ham,"said kiss, kiss, i can't do the sound effects..."
971,ham,MY NO. IN LUTON 0125698789 RING ME IF UR AROUN...
255,spam,Todays Voda numbers ending 1225 are selected t...
1298,ham,Ya very nice. . .be ready on thursday


## Create the word frequency dictionary for each class

Create a word frequency dictionary for each class.

In [39]:
#Your code here
class_word_freq = {} #Will be a nested dictionary of class_i : {word1:freq, word2:freq..., wordn:freq},.... class_m : {}
classes = train_df.label.unique()
for class_ in classes:
    temp_df = train_df[train_df.label==class_]
    bag = {}
    for row in temp_df.index:
        doc = temp_df['text'][row]
        for word in doc.split():
            bag[word] = bag.get(word, 0) + 1
    class_word_freq[class_] = bag
class_word_freq['spam']

{'Todays': 8,
 'Voda': 3,
 'numbers': 4,
 'ending': 4,
 '1225': 1,
 'are': 37,
 'selected': 15,
 'to': 233,
 'receive': 18,
 'a': 130,
 '£50award.': 1,
 'If': 5,
 'you': 63,
 'have': 34,
 'match': 3,
 'please': 9,
 'call': 57,
 '08712300220': 3,
 'quoting': 3,
 'claim': 23,
 'code': 6,
 '3100': 2,
 'standard': 4,
 'rates': 5,
 'app': 2,
 'Great': 1,
 'NEW': 6,
 'Offer': 1,
 '-': 16,
 'DOUBLE': 2,
 'Mins': 3,
 '&': 32,
 'Txt': 22,
 'on': 33,
 'best': 3,
 'Orange': 4,
 'tariffs': 1,
 'AND': 2,
 'get': 16,
 'latest': 11,
 'camera': 5,
 'phones': 4,
 '4': 34,
 'FREE!': 2,
 'Call': 38,
 'MobileUpd8': 3,
 'free': 22,
 '08000839402': 3,
 'NOW!': 3,
 'or': 63,
 '2stoptxt': 1,
 'T&Cs': 9,
 'You': 24,
 'can': 6,
 'donate': 1,
 '£2.50': 2,
 "UNICEF's": 1,
 'Asian': 1,
 'Tsunami': 1,
 'disaster': 1,
 'support': 2,
 'fund': 1,
 'by': 14,
 'texting': 1,
 'DONATE': 1,
 '864233.': 1,
 'will': 18,
 'be': 15,
 'added': 1,
 'your': 70,
 'next': 5,
 'bill': 2,
 'For': 8,
 'ur': 42,
 'chance': 9,
 'win': 1

## Count the Total Corpus Words
Calculate V, the total number of words in the corpus.

In [40]:
#Your code here
vocabulary = set()
for text in train_df.text:
    for word in text.split():
        vocabulary.add(word)
V = len(vocabulary)
V

3528

## Create a Bag of Words Function

Before implementing the entire Naive Bayes algorithm, create a helper function `bag_it()` to create a bag of words representation from a document's text.

In [41]:
#Your code here
def bag_it(doc):
    bag = {}
    for word in doc.split():
        bag[word] = bag.get(word, 0) + 1
    return bag

## Implementing Naive Bayes

Now, implement a master function to build a naive Bayes classifier. Be sure to use the logarithmic probabilities to avoid underflow.

In [52]:
p_classes = dict(train_df.label.value_counts(normalize=True))
p_classes

{'spam': 0.5030425963488844, 'ham': 0.4969574036511156}

In [50]:
import numpy as np

#Your code here
def classify_doc(doc, class_word_freq, p_classes, V, return_posteriors=False):
    bag = bag_it(doc)
    classes = []
    posteriors = []
    for class_ in class_word_freq.keys():
        p = np.log(p_classes[class_])
        for word in bag.keys():
            num = bag[word]+1
            denom = class_word_freq[class_].get(word, 0) + V
            p += np.log(num/denom)
        classes.append(class_)
        posteriors.append(p)
    if return_posteriors:
        print(posteriors)
    return classes[np.argmax(posteriors)]

In [53]:
train_df.iloc[0]

label                          ham
text     Its a part of checking IQ
Name: 797, dtype: object

In [51]:
classify_doc(train_df.iloc[0]['text'], class_word_freq, p_classes, V, return_posteriors=True)

[-45.58112560245864, -45.58630360936799]


'ham'

## Test Out Your Classifier

Finally, test out your classifier and measure its accuracy. Don't be perturbed if your results are sub-par; industry use cases would require substantial additional preprocessing before implementing the algorithm in practice.

In [ ]:
#Your Code here
train_label_hat = train_df.map(lambda entry: classify_doc(entry['text'], class_word_freq, p_classes, V, True), )

## Level-Up

Rework your code into an appropriate class structure so that you could easily implement the algorithm on any given dataset.

## Summary

Well done! In this lab, you practiced implementing naive Bayes' for document classification!